In [ ]:
!wget https://openi.nlm.nih.gov/imgs/collections/NLMCXR_png.tgz
!wget https://openi.nlm.nih.gov/imgs/collections/NLMCXR_reports.tgz

In [ ]:
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
#from tqdm.notebook import tqdm
import os
import re
import seaborn as sns
from tqdm import tqdm_notebook as tqdm
import warnings
warnings.filterwarnings('ignore')
from collections import defaultdict
#import tensorflow as tf

In [ ]:
import tarfile

file_path = '/content/NLMCXR_png.tgz'  # Replace with the actual path to your file

with tarfile.open(file_path, 'r:gz') as tar:
    tar.extractall('/content/images')  # Replace with the desired extraction directory


In [ ]:
import tarfile
file_path = '/content/NLMCXR_reports.tgz'  # Replace with the actual path to your file
with tarfile.open(file_path, 'r:gz') as tar:
    tar.extractall('/content/reports')  # Replace with the desired extraction directory


In [ ]:
columns = ["image_id", "caption", "comparison", "indication", "findings", "impression","height","width"]
df = pd.DataFrame(columns = columns)

In [ ]:
len(os.listdir("/content/images"))

In [ ]:
import tarfile
import os
import pandas as pd
for file in tqdm(os.listdir("/content/reports/ecgen-radiology/")):
    if file.endswith(".xml"):
        k = "/content/reports/ecgen-radiology/"
        path = k + file
        mytree = ET.parse(path)# parsing xml report
        comparision = mytree.find(".//AbstractText[@Label='COMPARISON']").text # extracting comaparison text
        indication = mytree.find(".//AbstractText[@Label='INDICATION']").text #extracting indication text
        findings = mytree.find(".//AbstractText[@Label='FINDINGS']").text # extracting findings text
        impression = mytree.find(".//AbstractText[@Label='IMPRESSION']").text  # extracting impression text

        mytree = ET.parse(path)
        for x in mytree.findall("parentImage"):
            image_id = x.attrib['id']+".png"
            filename = '/content/images/' + image_id
            image = cv2.imread(filename) # reading image

            height, width, channels = image.shape
            caption = '' if x.find('caption').text is None else x.find('caption').text

            df = df.append(pd.Series([image_id, caption, comparision, indication, findings, impression,height,width],
                                                         index = columns), ignore_index = True)

In [ ]:
df.head(2)

In [ ]:
df.shape

In [ ]:
df[df['image_id']=='CXR1005_IM-0006-3003.png']

In [ ]:
df.head(5)

In [ ]:
def absolute_path(x):
    '''Makes the path absolute '''
    x = '/content/images/'+ x
    return x

df['Image_path'] = df['image_id'].apply(lambda x : absolute_path(x)) # making the paths absolute

In [ ]:
df.head(2)

In [ ]:
# Rename columns
df = df.rename(columns={
    'image_id': 'image_id',
    'eCitation': 'caption',
    'COMPARISON': 'comparison',
    'INDICATION': 'indication',
    'FINDINGS': 'findings',
    'IMPRESSION': 'impression',
    'height': 'height',
    'width': 'width',
    'Image_path': 'Image_path'
})

# Optional: Reorder columns
df = df[['image_id', 'caption', 'comparison', 'indication', 'findings', 'impression', 'height', 'width', 'Image_path']]
df.head(2)

In [ ]:
df[df['image_id']=='CXR1_1_IM-0001-3001.png']

In [ ]:
df.info()

In [ ]:
count = 1
fig = plt.figure(figsize=(15, 35))

for filename in df['Image_path'].values[95:100]:
    findings = list(df["findings"].loc[df["Image_path"] == filename].values)

    img = cv2.imread(filename)
    if img is None:
        print(f"Error loading image: {filename}")
        continue  # Skip to the next iteration if the image cannot be loaded

    ax = fig.add_subplot(5, 2, count, xticks=[], yticks=[])
    ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))  # Convert BGR to RGB
    count += 1

    ax = fig.add_subplot(5, 2, count)
    plt.axis('off')
    ax.plot()
    ax.set_xlim(0, 1)
    ax.set_ylim(0, len(findings))

    for i, f in enumerate(findings):
        ax.text(0, i, f, fontsize=20)

    count += 1

plt.show()


In [ ]:
plt.figure(figsize=(8,7))
img = cv2.imread(df['Image_path'].values[20])
plt.imshow(img)
plt.title(df['Image_path'].values[20])

df['findings'].values[20]

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Plotting height
plt.figure(figsize=(10, 4))
plt.subplot(121)
plt.title('Height Plot')
plt.ylabel('Heights')
plt.xlabel('--Images--')
sns.scatterplot(x=range(len(df.height.values)), y=df.height.values)

# Plotting width
plt.subplot(122)
plt.title('Width Plot')
plt.ylabel('Widths')
plt.xlabel('--Images--')
sns.scatterplot(x=range(len(df.width.values)), y=df.width.values)

plt.tight_layout()
plt.show()


In [ ]:
# number of missing values
df.isnull().sum()

In [ ]:
data = df[['image_id','findings','height','width','Image_path']]

In [ ]:
data.isnull().sum()

In [ ]:
data = data.dropna(axis=0) # drop all missing value rows

In [ ]:
plt.figure(figsize=(8,7))
img = cv2.imread(data['Image_path'].values[5])
plt.imshow(img)
plt.title(data['Image_path'].values[5])

data['findings'].values[5]

In [ ]:
data.Image_path

In [ ]:
data.head(1)

In [ ]:
images = {}
findings = {}

# Assuming `data` is a DataFrame with columns 'Image_path' and 'findings'
for img, fin in data[['Image_path', 'findings']].values:
    # Split the image path to get the filename and extract information
    a = img.split('.')
    file_type = a[-1]  # Get the file type (e.g., 'png')

    # Further split the filename to extract relevant information
    a = a[0].split('-')
    a.pop(len(a)-1)
    a = '-'.join(e for e in a)  # Join the remaining parts with '-'

    # Check if the key already exists in the 'images' dictionary
    if a not in images.keys():
        # If not, create a new entry with count 1 and store findings
        images[a] = 1
        findings[a] = fin
    else:
        # If the key already exists, increment the count and update findings
        images[a] += 1
        findings[a] = fin


In [ ]:
images

In [ ]:

images['/content/images/CXR368_IM-1832'],findings['/content/images/CXR368_IM-1832']

In [ ]:
images['/content/images/CXR305_IM-1420'],findings['/content/images/CXR305_IM-1420']

In [ ]:
print('Total Number of Unique_IDs :', len(images.keys()))

In [ ]:
plt.title('Number of Images  per patients')
sns.countplot(list(images.values()))

In [ ]:
def train_test_split(data):
    persons = list(data.keys())
    persons_train = persons[:2500]
    persons_cv = persons[2500:3000]
    persons_test = persons[3000:3350]
    return persons_train, persons_cv, persons_test

images_train, images_cv, images_test = train_test_split(images)

In [ ]:
images_train

In [ ]:
def combining_images(image_set):

    image_per_person = defaultdict(list)  # creating a list of dictionary to store all the image paths
                                            #corresponding to a person_id
    for pid in image_set:
        for img in data['Image_path'].values:
            if pid in img:
                image_per_person[pid].append(img)
            else:
                continue
    return image_per_person

In [ ]:
img_per_person_train = combining_images(images_train)
img_per_person_cv = combining_images(images_cv)
img_per_person_test = combining_images(images_test)

In [ ]:
img_per_person_train

In [ ]:
img_per_person_train['/content/images/CXR427_IM-2070']

In [ ]:
def load_image(file):
    img = cv2.imread(file)
    return img

In [ ]:
# just checking the ID which has 4 images
for k,v in images.items():
    if v == 4:
        print(k)
        break


In [ ]:
img_per_person_train

In [ ]:
plt.figure(figsize=(9,9))
plt.subplot(221)
plt.imshow(load_image('/content/images/CXR2646_IM-1131-1001.png'))
plt.title('/content/images/CXR2646_IM-1131-1001.png')
plt.subplot(222)
plt.imshow(load_image('/content/images/CXR1001_IM-0004-1001.png'))
plt.title('/content/images/CXR1001_IM-0004-1001.png')
plt.subplot(223)
plt.imshow(load_image('/content/images/CXR1003_IM-0005-2002.png'))
plt.title('/content/images/CXR1003_IM-0005-2002.png')
plt.subplot(224)
plt.imshow(load_image('/content/images/CXR1005_IM-0006-1001.png'))
plt.title('/content/images/CXR1005_IM-0006-1001.png')

In [ ]:
import itertools

def create_data(image_per_person):
    # new dataset
    person_id, image1, image2, report = [],[],[],[]
    for pid, imgs in image_per_person.items():   #contains pid and the images associated with that pid

        if len(imgs) == 1:
            image1.append(imgs[0])
            image2.append(imgs[0])
            person_id.append(pid)
            report.append(findings[pid])
        else:
            num = 0
            a = itertools.combinations(imgs, 2)
            for i in a:
                image1.append(i[0])
                image2.append(i[1])
                person_id.append(pid + '_' + str(num))
                report.append(findings[pid])
                num += 1
    data = pd.DataFrame()
    data['Person_id'] = person_id
    data['Image1'] = image1
    data['Image2'] = image2
    data['Report'] = report

    return data

In [ ]:
train = create_data(img_per_person_train)
test = create_data(img_per_person_test)
cv = create_data(img_per_person_cv)

In [ ]:
train.head(2)

In [ ]:
train.to_csv('train.csv')
test.to_csv('test.csv')
cv.to_csv('cv.csv')

#Preporcessing text data¶

In [ ]:
def lowercase(text):
    '''Converts to lowercase'''
    new_text = []
    for line in text:
        new_text.append(line.lower())
    return new_text

def decontractions(text):
    '''Performs decontractions in the doc'''
    new_text = []
    for phrase in text:
        phrase = re.sub(r"won't", "will not", phrase)
        phrase = re.sub(r"can\'t", "can not", phrase)
        phrase = re.sub(r"couldn\'t", "could not", phrase)
        phrase = re.sub(r"shouldn\'t", "should not", phrase)
        phrase = re.sub(r"wouldn\'t", "would not", phrase)
        # general
        phrase = re.sub(r"n\'t", " not", phrase)
        phrase = re.sub(r"\'re", " are", phrase)
        phrase = re.sub(r"\'s", " is", phrase)
        phrase = re.sub(r"\'d", " would", phrase)
        phrase = re.sub(r"\'ll", " will", phrase)
        phrase = re.sub(r"\'t", " not", phrase)
        phrase = re.sub(r"\'ve", " have", phrase)
        phrase = re.sub(r"\'m", " am", phrase)
        phrase = re.sub(r"\*+", "abuse", phrase)
        new_text.append(phrase)

    return new_text

def rem_punctuations(text):
    '''Removes punctuations'''
    punctuations = '''!()-[]{};:'"\,<>/?@#$%^&*~''' # full stop is not removed
    new_text = []
    for line in text:
        for char in line:
            if char in punctuations:
                line = line.replace(char, "")
        new_text.append(' '.join(e for e in line.split()))
    return new_text

def rem_numbers(text):
    '''Removes numbers and irrelevant text like xxxx*'''
    new_text = []
    for line in text:
        temp = re.sub(r'x*','',line)
        new_text.append(re.sub(r'\d','',temp))
    return new_text

def words_filter(text):
    '''Removes words less than 2 characters except no and ct'''
    new_text = []
    for line in text:
        temp = line.split()
        temp2 = []
        for word in temp:
            if  len(word) <=2 and word != 'no' and word != 'ct':
                continue
            else:
                temp2.append(word)
        new_text.append(' '.join(e for e in temp2))
    return new_text

def multiple_fullstops(text):
    ''' Removes multiple full stops from the text'''
    new_text = []
    for line in text:
        new_text.append(re.sub(r'\.\.+', '.', line))
    return new_text

def fullstops(text):
    new_text = []
    for line in text:
        new_text.append(re.sub('\.', ' .', line))
    return new_text

def multiple_spaces(text):
    new_text = []
    for line in text:
        new_text.append(' '.join(e for e in line.split()))
    return new_text

def separting_startg_words(text):
    new_text = []
    for line in text:
        temp = []
        words = line.split()
        for i in words:
            if i.startswith('.') == False:
                temp.append(i)
            else:
                w = i.replace('.','. ')
                temp.append(w)
        new_text.append(' '.join(e for e in temp))
    return new_text

def rem_apostrophes(text):
    new_text = []
    for line in text:
        new_text.append(re.sub("'",'',line))
    return new_text

In [ ]:
def text_preprocessing(text):
    '''Combines all the preprocess functions'''
    new_text = lowercase(text)
    new_text = decontractions(new_text)
    new_text = rem_punctuations(new_text)
    new_text = rem_numbers(new_text)
    new_text = words_filter(new_text)
    new_text = multiple_fullstops(new_text)
    new_text = fullstops(new_text)
    new_text = multiple_spaces(new_text)
    new_text = separting_startg_words(new_text)
    new_text = rem_apostrophes(new_text)
    return new_text

In [ ]:
train['Report'] = text_preprocessing(train['Report'])
test['Report'] = text_preprocessing(test['Report'])
cv['Report'] = text_preprocessing(cv['Report'])

In [ ]:
train

In [ ]:
train.to_csv('processed_train.csv')
test.to_csv('processed_test.csv')
cv.to_csv('processed_cv.csv')

In [ ]:
l = [len(e.split()) for e in train['Report'].values]  # Number of words in each report

In [ ]:
print('maximum word in a report is :',max(l))

In [ ]:
plt.title('Number of Words per Report')
sns.scatterplot(x=range(train.shape[0]), y=l)
plt.ylabel('Number of words')
plt.show()


In [ ]:
from wordcloud import WordCloud
def show_wordcloud(data, title = None):
    wordcloud = WordCloud(background_color='black',max_words=800,max_font_size=60,scale=3,random_state=1 ).generate(' '.join(data.astype(str)))

    fig = plt.figure(1, figsize=(12, 15))
    plt.axis('off')
    if title:
        fig.suptitle(title, fontsize=20)
        fig.subplots_adjust(top=2.3)

    plt.imshow(wordcloud)
    plt.show()

show_wordcloud(train['Report'])

In [ ]:
countword = train['Report'].str.split().apply(len).value_counts()
countword[:].plot(kind='bar',figsize=(20,5) , title = 'Words for each findings review')

In [ ]:
def remodelling(x):
    '''adds start and end tokens to a sentence '''
    return 'startseq' + ' ' + x + ' ' + 'endseq'

In [ ]:
train['Report'] = train['Report'].apply(lambda x : remodelling(x))
test['Report'] = test['Report'].apply(lambda x : remodelling(x))
cv['Report'] = cv['Report'].apply(lambda x : remodelling(x))

In [ ]:
train['Report'][1]

In [ ]:
# save the cleaned data(STRUCTURED DATA)
train.to_csv('Final_Train_Data.csv', index=False)
test.to_csv('Final_Test_Data.csv', index=False)
cv.to_csv('Final_CV_Data.csv', index=False)

In [ ]:
train

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import densenet
from tensorflow.keras.applications.densenet import preprocess_input
from tensorflow.keras.layers import Dense, Dropout, Input, Conv2D
from tensorflow.keras.models import Model
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import os
import cv2
import tensorflow as tf
import re
import pickle
from PIL import Image
from skimage.transform import resize
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import time
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, LSTM, Input, Embedding, Conv2D, Concatenate, Flatten, Add, Dropout, GRU
import random
import datetime
from nltk.translate.bleu_score import sentence_bleu

In [ ]:
chexNet = densenet.DenseNet121(include_top=False, weights = None,   input_shape=(224,224,3), pooling="avg")
X = chexNet.output
X = Dense(14, activation="sigmoid", name="predictions")(X)
model = Model(inputs=chexNet.input, outputs=X)

! gdown "https://drive.google.com/u/0/uc?id=19BllaOvs2x5PLV_vlWMy4i8LapLb2j6b&export=download"



In [ ]:
#loadind pretrained weights for ChexNet model
model.load_weights('/content/brucechou1983_CheXNet_Keras_0.3.0_weights.h5')

chexNet = Model(inputs = model.input, outputs = model.layers[-2].output)

In [ ]:
import pandas as pd

train=pd.read_csv("/content/Final_Train_Data.csv")
test=pd.read_csv("/content/Final_Test_Data.csv")
cv=pd.read_csv("/content/Final_CV_Data.csv")

In [ ]:
train

In [ ]:
from keras.applications.densenet import preprocess_input
from keras.preprocessing.image import img_to_array, load_img
from keras.models import Model
from keras.layers import Dense
from tqdm import tqdm
import numpy as np
import pandas as pd

# Assuming 'train', 'test', and 'cv' are your DataFrames containing image paths and findings

def load_and_preprocess_image(image_path):
    # Load the image and resize it to (224, 224)
    img = load_img(image_path, target_size=(224, 224))
    # Convert the image to a numpy array
    img_array = img_to_array(img)
    # Expand the dimensions to match the expected input shape
    img_array = np.expand_dims(img_array, axis=0)
    # Preprocess the image (e.g., normalize pixel values)
    img_array = preprocess_input(img_array)
    return img_array

# Load the pre-trained ChexNet model
chexNet = densenet.DenseNet121(include_top=False, weights=None, input_shape=(224, 224, 3), pooling="avg")
X = chexNet.output
X = Dense(14, activation="sigmoid", name="predictions")(X)
model = Model(inputs=chexNet.input, outputs=X)

# Load the pre-trained weights for the ChexNet model
!gdown "https://drive.google.com/u/0/uc?id=19BllaOvs2x5PLV_vlWMy4i8LapLb2j6b&export=download"
model.load_weights('/content/brucechou1983_CheXNet_Keras_0.3.0_weights.h5')

# Create a modified ChexNet model for feature extraction
chexNet = Model(inputs=model.input, outputs=model.layers[-2].output)

Xnet_features = {}

# Process the 'train' dataset
for key, img1, img2, finding in tqdm(train.values):
    i1 = load_and_preprocess_image(img1)
    img1_features = chexNet.predict(i1)
    i2 = load_and_preprocess_image(img2)
    img2_features = chexNet.predict(i2)
    input_ = np.concatenate((img1_features, img2_features), axis=1)
    Xnet_features[key] = input_

# Process the 'test' dataset
for key, img1, img2, finding in tqdm(test.values):
    i1 = load_and_preprocess_image(img1)
    img1_features = chexNet.predict(i1)
    i2 = load_and_preprocess_image(img2)
    img2_features = chexNet.predict(i2)
    input_ = np.concatenate((img1_features, img2_features), axis=1)
    Xnet_features[key] = input_

# Process the 'cv' dataset
for key, img1, img2, finding in tqdm(cv.values):
    i1 = load_and_preprocess_image(img1)
    img1_features = chexNet.predict(i1)
    i2 = load_and_preprocess_image(img2)
    img2_features = chexNet.predict(i2)
    input_ = np.concatenate((img1_features, img2_features), axis=1)
    Xnet_features[key] = input_


In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

In [ ]:
!unzip /content/glove.6B.zip

In [ ]:
X_train = train['Person_id']
X_test = test['Person_id']
X_cv = cv['Person_id']
y_train = train['Report']
y_test = test['Report']
y_cv = cv['Report']

cheXnet_Features = Xnet_features

#Now we tokenize our text data and create a emending matrix.

In [ ]:
from tqdm import tqdm
import numpy as np
from keras.preprocessing.text import Tokenizer
import pickle

# Assuming 'y_train' is a Series containing your text data

# Initialize Tokenizer
tokenizer = Tokenizer(filters='!"#$%&()*+,-/:;<=>?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(y_train.values)

# Set padding size
padding_size = 153   # Max length

# Calculate vocabulary size
vocab_size = len(tokenizer.word_index.keys()) + 1

# Load GloVe vectors from a text file
glove_vectors = {}
with open('/content/glove.6B.300d.txt', 'r', encoding='utf-8') as f:
    for line in tqdm(f, total=400000):
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], dtype='float32')
        glove_vectors[word] = vector

# Create an embedding matrix
embedding_matrix = np.zeros((vocab_size, 300))
for word, i in tokenizer.word_index.items():
    if word in glove_vectors:
        vec = glove_vectors[word]
        embedding_matrix[i] = vec
    else:
        continue


In [ ]:
glove_vectors


#8.4 Create Dataset generator
We need to pass image features for two images as well as the the actual report to our model for training. Thus we create a data generator function to serve our purpose.

In [ ]:
BATCH_SIZE = 32
def load_image(id_, report):
    '''Loads the Image Features with their corresponding Ids'''
    img_feature = cheXnet_Features[id_.decode('utf-8')][0]
    return img_feature, report

def dataset_generator(img_name, caption):
    dataset = tf.data.Dataset.from_tensor_slices((img_name, caption))

    # Use map to load the numpy files in parallel
    dataset = dataset.map(lambda item1, item2: tf.numpy_function(load_image, [item1, item2], [tf.float32, tf.string]),
                          num_parallel_calls=tf.data.experimental.AUTOTUNE)

    # Shuffle and batch
    dataset = dataset.shuffle(500).batch(BATCH_SIZE).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    return dataset

train_generator = dataset_generator(X_train, y_train)
cv_generator = dataset_generator(X_cv, y_cv)


The generator gives provides data in bytes. We create a function that converts them back to strings for manipulation. Also we create a function that takes a batch of data and converts them into new data frame.

In [ ]:
def bytes_to_string(arr):
    '''The generator gives provides data in bytes. This function converts them back to strings for manipulation'''
    for i in range(len(arr)):
        arr[i] = arr[i].decode('utf-8')
    return arr

def convert(images, reports):
    '''This function takes the batch of data and converts them into a new dataset'''
    imgs = []
    in_reports = []
    out_reports = []
    for i in range(len(images)):
        sequence = [tokenizer.word_index[e] for e in reports[i].split() if e in tokenizer.word_index.keys()]
        #  print(sequence)
        for j in range(1,len(sequence)):
            in_seq = sequence[:j]
            out_seq = sequence[j]
            out_seq = tf.keras.utils.to_categorical(out_seq, num_classes=vocab_size)

            imgs.append(images[i])
            #  print(in_seq)
            in_reports.append(in_seq)
            # print(out_seq)
            out_reports.append(out_seq)

    return np.array(imgs), np.array(in_reports), np.array(out_reports)


8.5 Defining our final model
Now Since we are equipped with all the necessary utility function and image feature we finally define our sequence to sequence model.

In [ ]:
input1 = Input(shape=(2048), name='Image_input')
dense1 = Dense(256, kernel_initializer=tf.keras.initializers.glorot_uniform(seed = 56), name='dense_encoder')(input1)

input2 = Input(shape=(153), name='Text_Input')
embedding_layer = Embedding(input_dim = vocab_size, output_dim = 300, input_length=153, mask_zero=True, trainable=False,
                weights=[embedding_matrix], name="Embedding_layer")
emb = embedding_layer(input2)

LSTM1 = LSTM(units=256, activation='tanh', recurrent_activation='sigmoid', use_bias=True,
            kernel_initializer=tf.keras.initializers.glorot_uniform(seed=23),
            recurrent_initializer=tf.keras.initializers.orthogonal(seed=7),
            bias_initializer=tf.keras.initializers.zeros(), return_sequences=True, name="LSTM1")(emb)
#LSTM1_output = LSTM1(emb)

LSTM2 = LSTM(units=256, activation='tanh', recurrent_activation='sigmoid', use_bias=True,
            kernel_initializer=tf.keras.initializers.glorot_uniform(seed=23),
            recurrent_initializer=tf.keras.initializers.orthogonal(seed=7),
            bias_initializer=tf.keras.initializers.zeros(), name="LSTM2")
LSTM2_output = LSTM2(LSTM1)

dropout1 = Dropout(0.5, name='dropout1')(LSTM2_output)

dec =  tf.keras.layers.Add()([dense1, dropout1])

fc1 = Dense(256, activation='relu', kernel_initializer=tf.keras.initializers.he_normal(seed = 63), name='fc1')
fc1_output = fc1(dec)
dropout2 = Dropout(0.4, name='dropout2')(fc1_output)
output_layer = Dense(vocab_size, activation='softmax', name='Output_layer')
output = output_layer(dropout2)

encoder_decoder = Model(inputs = [input1, input2], outputs = output)
encoder_decoder.summary()

8.6 Defining loss function
We defined Masked Loss Function for this problem. For eg: If we have a sequence of tokens- [3],[10],[7],[0],[0],[0],[0],[0].We only have 3 words in this sequence, the zeros correspond to the padding which is actually not a part of the report. But the model will think that the zeros are also a part of the sequence and will start learning them. When the model starts to correctly predict the zeros, the loss will decrease because for the model it is learning correctly. But for us the loss should only decrease if the model is predicting the actual words(non-zeros) correctly.

Therefore we should mask the zeros in the sequence so that the model don’t give its attention to them and only learns the needed words in the report.

In [ ]:
loss_function = tf.keras.losses.CategoricalCrossentropy(from_logits=False, reduction='auto')

def maskedLoss(y_true, y_pred):
    #getting mask value
    mask = tf.math.logical_not(tf.math.equal(y_true, 0))

    #calculating the loss
    loss_ = loss_function(y_true, y_pred)

    #converting mask dtype to loss_ dtype
    mask = tf.cast(mask, dtype=loss_.dtype)

    #applying the mask to loss
    loss_ = loss_*mask

    #getting mean over all the values
    loss_ = tf.reduce_mean(loss_)
    return loss_

8.7 Model Training
Now its time to compile and train our model on the dataset.

We are going to use Adam Optimizer for faster convergence. The Adam optimization algorithm is an extension to stochastic gradient descent that has recently seen broader adoption for deep learning applications in computer vision and natural language processing.

Adam as combining the advantages of two other extensions of stochastic gradient descent. Specifically:

Adaptive Gradient Algorithm (AdaGrad) that maintains a per-parameter learning rate that improves performance on problems with sparse gradients (e.g. natural language and computer vision problems).
Root Mean Square Propagation (RMSProp) that also maintains per-parameter learning rates that are adapted based on the average of recent magnitudes of the gradients for the weight (e.g. how quickly it is changing). This means the algorithm does well on online and non-stationary problems (e.g. noisy).
Adam realizes the benefits of both AdaGrad and RMSProp.

Instead of adapting the parameter learning rates based on the average first moment (the mean) as in RMSProp, Adam also makes use of the average of the second moments of the gradients (the uncentered variance).

In [ ]:
optimizer = tf.keras.optimizers.Adam(0.001)
encoder_decoder.compile(optimizer, loss = maskedLoss)

current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'Tensorboard/logs_m1/fit3/' + current_time + '/train'
val_log_dir = 'Tensorboard/logs_m1/fit3/' + current_time + '/test'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
val_summary_writer = tf.summary.create_file_writer(val_log_dir)

#training for 20 epochs
epoch_train_loss = []
epoch_val_loss = []

for epoch in range(20):
    print('EPOCH : ',epoch+1)
    start = time.time()
    batch_loss_tr = 0
    batch_loss_vl = 0

    for img, report in train_generator:

        r1 = bytes_to_string(report.numpy())
        img_input, rep_input, output_word = convert(img.numpy(), r1)
        rep_input = pad_sequences(rep_input, maxlen=153, padding='post')
        results = encoder_decoder.train_on_batch([img_input, rep_input], output_word)

        batch_loss_tr += results

    train_loss = batch_loss_tr/(X_train.shape[0]//14)
 #   print('Saving Tensorboard')
    with train_summary_writer.as_default():
        tf.summary.scalar('loss', train_loss, step = epoch)

    for img, report in cv_generator:

        r1 = bytes_to_string(report.numpy())
        img_input, rep_input, output_word = convert(img.numpy(), r1)
        rep_input = pad_sequences(rep_input, maxlen=153, padding='post')
        results = encoder_decoder.test_on_batch([img_input, rep_input], output_word)
        batch_loss_vl += results

    val_loss = batch_loss_vl/(X_cv.shape[0]//14)

    with val_summary_writer.as_default():
        tf.summary.scalar('loss', val_loss, step = epoch)

    epoch_train_loss.append(train_loss)

    epoch_val_loss.append(val_loss)

    print('Training Loss: {},  Val Loss: {}'.format(train_loss, val_loss))
    print('Time Taken for this Epoch : {} sec'.format(time.time()-start))
    encoder_decoder.save_weights('encoder_decoder_epoch_'+ str(epoch+1) + '.h5')
    print('--------------------------------------------------------------------------------------------')

In [ ]:
import matplotlib.pyplot as plt


# Plotting the results
plt.plot(range(1, 21), epoch_train_loss, label='Training Loss')
plt.plot(range(1, 21), epoch_val_loss, label='Validation Loss')
plt.title('Training and Validation Loss Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


8.8 Interface
Now that we have trained our model, it’s time to prepare our model to predict reports. For this purpose we have to make some adjustments in our model. This will save us some time during testing. First we will separate the encoder and decoder part from our model. The features predicted by the encoder will be used as the input to our decoder along with the partial reports.

In [ ]:
# encoder
encoder_input = encoder_decoder.input[0]
encoder_output = encoder_decoder.get_layer('dense_encoder').output
encoder_model = Model(encoder_input, encoder_output)

# decoder#
text_input = encoder_decoder.input[1]
enc_output = Input(shape=(256,), name='Enc_Output')
text_output = encoder_decoder.get_layer('LSTM2').output
add1 = tf.keras.layers.Add()([text_output, enc_output])
fc_1 = fc1(add1)
decoder_output = output_layer(fc_1)

decoder_model = Model(inputs = [text_input, enc_output], outputs = decoder_output)

To predict the report form our model given the images, we will use greedy search and beam search algorithm.

8.9 Greedy search
Greedy is an algorithmic paradigm that builds up a solution piece by piece, always choosing the next piece that offers the most obvious and immediate benefit. So the problems where choosing locally optimal also leads to global solution are best fit for Greedy.

A simple approximation is to use a greedy search that selects the most likely word at each step in the output sequence. This approach has the benefit that it is very fast, but the quality of the final output sequences may be far from optimal.

In [ ]:
def greedysearch(img):
    image = cheXnet_Features[img]
    input_ = 'startseq'
    image_features = encoder_model.predict(image)

    result = []
    for i in range(153):
        input_tok = [tokenizer.word_index[w] for w in input_.split()]
        input_padded = pad_sequences([input_tok], 153, padding='post')
        predictions = decoder_model.predict([input_padded, image_features])
        arg = np.argmax(predictions)
        if arg != 7:   # endseq
            result.append(tokenizer.index_word[arg])
            input_ = input_ + ' ' + tokenizer.index_word[arg]
        else:
            break
    rep = ' '.join(e for e in result)
    return rep

Lets define a function that takes index of an image as input and returns both actual and predicted report using greedy search.

In [ ]:
cv_data =pd.read_csv("/content/Final_CV_Data.csv")


In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.applications.inception_v3 import preprocess_input
from skimage.transform import resize

def load_image(img_name):
    image = Image.open(img_name)
    X = np.asarray(image.convert("RGB"))
    X = np.asarray(X)
    X = preprocess_input(X)
    X = resize(X, (224, 224, 3))
    X = np.expand_dims(X, axis=0)
    X = np.asarray(X)
    return X

def get_result(idx=0):
    plt.figure(figsize=(9, 5))

    pre_Report = greedysearch(cv_data['Person_id'][idx])  # result after 20 epochs
    print('------------------------------------------------------------------------------------------------------')
    print("Predicted Report : ", pre_Report)
    print('------------------------------------------------------------------------------------------------------')
    print("Actual Report : ", cv_data['Report'][idx])

    plt.subplot(121)
    img = load_image(cv_data['Image1'][idx])
    plt.imshow(img[0])

    plt.subplot(122)
    img = load_image(cv_data['Image2'][idx])
    plt.imshow(img[0])




In [ ]:
get_result(2)

In [ ]:
def get_results(start_idx, end_idx):
    for idx in range(start_idx, end_idx):
        plt.figure(figsize=(9, 5))

        pre_Report = greedysearch(cv_data['Person_id'][idx])  # result after 20 epochs
        print('------------------------------------------------------------------------------------------------------')
        print("Predicted Report : ", pre_Report)
        print('------------------------------------------------------------------------------------------------------')
        print("Actual Report : ", cv_data['Report'][idx])

        plt.subplot(121)
        img = load_image(cv_data['Image1'][idx])
        plt.imshow(img[0])

        plt.subplot(122)
        img = load_image(cv_data['Image2'][idx])
        plt.imshow(img[0])

        plt.show()

# Make predictions for images with indices from 0 to 14
get_results(0, 15)


In [ ]:
get_result(2)

In [ ]:
# Save model weights
encoder_decoder.save_weights('encoder_decoder_weights.h5')




In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.applications.inception_v3 import preprocess_input
from skimage.transform import resize

def load_image(img_name):
    image = Image.open(img_name)
    X = np.asarray(image.convert("RGB"))
    X = np.asarray(X)
    X = preprocess_input(X)
    X = resize(X, (224, 224, 3))
    X = np.expand_dims(X, axis=0)
    X = np.asarray(X)
    return X

def get_result(idx=0):
    plt.figure(figsize=(9, 5))

    pre_Report = greedysearch(cv_data['Person_id'][idx])  # result after 20 epochs
    print('------------------------------------------------------------------------------------------------------')
    print("Predicted Report : ", pre_Report)
    print('------------------------------------------------------------------------------------------------------')
    print("Actual Report : ", cv_data['Report'][idx])

    plt.subplot(121)
    img = load_image(cv_data['Image1'][idx])
    plt.imshow(img[0])

    plt.subplot(122)
    img = load_image(cv_data['Image2'][idx])
    plt.imshow(img[0])


